In [47]:
# Preliminaries: Teach Julia that functions form a vector space
import Base.+,Base.*,Base.zero
+(f::Function,g::Function) = x -> f(x)+g(x)
*(c::Number,f::Function) = x -> c*f(x)
*(f::Function,c::Number) = x -> c*f(x)
zero(Function) = x -> 0

zero (generic function with 18 methods)

## Example 1: Derivatives of  (a cos x + b sin x) 
Consider the 2 dimensional vector space consisting of linear combinations of "sine" and "cosine". How can we take the derivative of a function in this vector space?

### Derivatives: Method 1, symbolically.  Matches the paper and pencil method closely.

In [26]:
using SymPy

In [39]:
x,a,b = Sym.(["x","a","b"]);

In [42]:
diff( a*cos(x) + b*sin(x) ,x)

-a*sin(x) + b*cos(x)

### Method 2, matrix-vector.  Emphasizes the linear nature of derivatives. (Easy to imagine a numerical implementation.)

$\begin{pmatrix} a' \\b' \end{pmatrix} = 
\begin{pmatrix} 0 & 1 \\-1 & 0 \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}$

Understanding: $\begin{pmatrix} a' \\b' \end{pmatrix}$ is shorthand for
$a\cos x + b\sin x$.

Example: Differentiate $f(x)=5\cos x + 2 \sin x$:
1. Encode  f(x) for computation as the vector  $\begin{pmatrix} 5 \\ 2 \end{pmatrix}.$
2. Apply $\begin{pmatrix} 0 & 1 \\-1 & 0 \end{pmatrix}$
to $\begin{pmatrix} 5 \\ 2 \end{pmatrix}$
yielding $\begin{pmatrix} 2 \\ -5 \end{pmatrix}.$
3. Decode $\begin{pmatrix} 2 \\ -5 \end{pmatrix}$ as
$2 \cos x -5 \sin x.$

###  Method 3: no shorthand.  Combines method 1 and method 2.

$\frac{d}{dx} \begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} 0 & -1 \\1 & 0 \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
$

Method 2 is purely numerical.  The interpretation is imposed by
the human.  Method 3 can be interpreted as method 2 (matrix times vector) with the labels. 

If one associates differently, Method 3 can be interpeted as knowing
the derivative on the basis functions is sufficient for knowing
this linear transformation everywhere:

$\frac{d}{dx} \begin{pmatrix} \sin x& \cos x \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} 0 & -1 \\1 & 0 \end{pmatrix}
$

### Observation:  
Method 1 is straightforward but in the end
bulky.  Method 2 shows that the linear transformation defined
by differentiating can be encoded as a simple matrix times vector,
which is very efficient on a computer, and also gets to the algebraic heart of the operation.  Method 3 organizes the symbolic with the matrices in a way that points to the generalization.
<br>
Most students of calculus learn that differentiation is linear.
Derivatives of sums are sums of derivatives ,(f+g)'=f'+g'.  Derivatives of
constant multiples are constant multiples of derivatives (cf)'=cf'.

### With code:

In [76]:
f=([sin cos]*[0 1;-1 0]*[5,2])[1]
x=rand()
[f(x) 2sin(x)-5cos(x)] ## Check that it gives the right function

1×2 Array{Float64,2}:
 -1.29521  -1.29521

## In general

If $v_1,\ldots,v_n$ is a basis for a vector space $V$, 
and <br> $w_1,\ldots,w_m$ is a basis for a vector space $W$,
and $T$ is some linear transformation,
we can write

$$ T[v_1,\ldots,v_n]
\begin{pmatrix} c_1 \\ \vdots \\ c_n \end{pmatrix}
=
[w_1,\ldots,w_m] * A* \begin{pmatrix} c_1 \\ \vdots  \\ c_n \end{pmatrix}$$
for some $m \times n$ matrix $A$.

One can associate the equation above concentrating
on
$$ T[v_1,\ldots,v_n]
=
[w_1,\ldots,w_m] * A$$


to think of
$T$ as applied to every basis vector of $V$ to get
some linear combination of the basis vectors of $W$,
or one can do "Method 2" and think of
$\begin{pmatrix} c_1 \\ \vdots \\ c_n \end{pmatrix}$
as the coefficients in the basis for $V$, and 
$A\begin{pmatrix} c_1 \\ \vdots \\ c_n \end{pmatrix}$
as the cooeficients of $Tv$ in the basis for $W$.

## Example 2:  Shifting (a cos x + b sin x)

Convince yourself without matrices that
$Tf$ defined by $ (Tf)(x)=f(x+\theta)$ is linear for
any constant $\theta$.

With matrices we have

$T  \begin{pmatrix} \sin x& \cos x \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} \cos \theta & -\sin \theta  \\ \sin \theta & \cos \theta \end{pmatrix}
$
or
$T  \begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
=
\begin{pmatrix} \sin x& \cos x \end{pmatrix}
\begin{pmatrix} \cos \theta & -\sin \theta  \\ \sin \theta & \cos \theta \end{pmatrix}
\begin{pmatrix} a \\b \end{pmatrix}
$


which can be done symbolically but gets a little messier looking.  The linear algebra is just tidier.

In [87]:
x = Sym("x")
f = a*sin(x) + b*cos(x)
Tf = subs(f,x,x+Sym("theta"))

a*sin(theta + x) + b*cos(theta + x)

In [90]:
expand_trig(Tf)

a*(sin(theta)*cos(x) + sin(x)*cos(theta)) + b*(-sin(theta)*sin(x) + cos(theta)
*cos(x))

Of course Example 1 is a special case of Example 2 because
the derivative is the same as shifting by $\pi/2$ on this very special  vector space.

## Example 3. Change of basis for polynomials

Suppose one wants to work with Laguerre polynomials.
Wikipidia can supply the first few of these for us:
 [Laguerre up to degree 6](https://en.wikipedia.org/wiki/Laguerre_polynomials#The_first_few_polynomials).

In [99]:
A = [
 1  -1   1   -1
 0   1  -4    9
 0   0   2  -18
 0   0   0    6]./[1 1 2 6]

4×4 Array{Float64,2}:
 1.0  -1.0   0.5  -0.166667
 0.0   1.0  -2.0   1.5     
 0.0   0.0   1.0  -3.0     
 0.0   0.0   0.0   1.0     

Check from the Wikipedia article that
$[L_0 \ L_1 \ L_2 \ L_3]=[1 \ x \ x^2 \ x^3] * A$

Convince yourself that to obtain
the coefficients of 
$c_0 L_0 + c_1 L_1 + c_2 L_2 + c_3 L_3$
in the standard basis $1,x,x^2,x^3$
one must simply compute
$A * \begin{pmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \end{pmatrix}.$

<br>

Notationally, we are saying that <br>
$[L_0 \ L_1 \ L_2 \ L_3]*\begin{pmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \end{pmatrix}=[1 \ x \ x^2 \ x^3] * A*\begin{pmatrix} c_0 \\ c_1 \\ c_2 \\ c_3 \end{pmatrix}$

Of course inv(A) let's us go the other way
(from monomial coefficients to Laguerre coefficients)

In [102]:
inv(A)

4×4 Array{Float64,2}:
 1.0  1.0  1.5  3.16667
 0.0  1.0  2.0  4.5    
 0.0  0.0  1.0  3.0    
 0.0  0.0  0.0  1.0    

Thus for example <br>
$x^4 = (19/6)L_0 + (9/2)L_1 + 3 L_2 + L_3.$

What if we want to differentiate quartics written in a Laguerre polynomial basis but we only know how to differentiate in a monomial basis?